In [ ]:
!pip install janome

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 56.9 MB/s eta 0:00:00


In [ ]:
import os
import glob
import re

In [ ]:
INPUT_DIR = "夏目漱石作品集"
os.makedirs(INPUT_DIR,exist_ok=True)

In [ ]:
raw_text = ""
file_paths = glob.glob(os.path.join(INPUT_DIR, '*.txt'))
total_files = len(file_paths)

if total_files == 0:
    print(f"エラー: '{INPUT_DIR}' ディレクトリ内に.txtファイルが見つかりませんでした。")
else:
    print(f"合計 {total_files} 個のテキストファイルを見つけました。")

合計 22 個のテキストファイルを見つけました。


# textの結合

In [ ]:
    # ファイルを一つずつ読み込み、内容を結合
    for file_path in file_paths:
        file_name = os.path.basename(file_path)
        try:
            # 青空文庫のファイルはShift_JISエンコーディングで開く
            with open(file_path, 'r', encoding='shift_jis') as f:
                content = f.read()
                raw_text += content
        except UnicodeDecodeError:
            print(f"警告: ファイル '{file_name}' のデコードに失敗しました。このファイルはスキップします。")
        except FileNotFoundError:
            # globで見つかっているはずなので通常は発生しない
            continue

    print(f"全作品の結合が完了しました。総文字数: {len(raw_text)}")

全作品の結合が完了しました。総文字数: 3117434


In [ ]:
cleaned_text = re.sub(r'《.*?》','',raw_text)

In [ ]:
cleaned_text = cleaned_text.replace("|","")
cleaned_text

In [ ]:
    cleaned_text = re.sub(r'［＃.*?］', '', cleaned_text, flags=re.DOTALL)
    cleaned_text = re.sub(r'［.*?］', '', cleaned_text) # 念のため残りの注釈も除去
    cleaned_text

In [ ]:
cleaned_text = re.sub(r'\n-{10,}\n.*?\n-{10,}\n', '\n', cleaned_text, flags=re.DOTALL)

In [ ]:
cleaned_text = re.sub(r'[\r\n\t\s　]+', ' ', cleaned_text).strip()

In [ ]:
cleaned_text = cleaned_text.replace('｜','').replace('※','')
cleaned_text


In [ ]:
punctuations = '「」『』（）【】、。！？・…ー'
for p in punctuations:
    cleaned_text = cleaned_text.replace(p, '')

In [ ]:
cleaned_text = cleaned_text.replace('夏目漱石','')
cleaned_text

In [ ]:
cleaned_text = re.sub(r'([一二三四五六七八九十]|[\d])[\s　]', '', cleaned_text)
cleaned_text

In [ ]:
OUTPUT_FILE = 'soseki_corpus_preprocessed.txt'
with open(OUTPUT_FILE, 'w', encoding='utf-8') as f:
    f.write(cleaned_text)

print(f"クリーンアップ済みのテキストを '{OUTPUT_FILE}' に保存しました。")

クリーンアップ済みのテキストを 'soseki_corpus_preprocessed.txt' に保存しました。


# 形態素解析

In [ ]:
from janome.tokenizer import Tokenizer

In [ ]:
t = Tokenizer()
parsed_words = []
a = "わたしは走っている犬を見ました"
TARGET_POS = ('名詞', '動詞', '形容詞', '副詞')

for token in t.tokenize(a):
    print(token.part_of_speech)
    print(type(token.part_of_speech.split(",")))
    pos = token.part_of_speech.split(',')[0]
    print(pos)

    if pos in TARGET_POS:
        word = token.base_form

        if word and word != '*':#word != "*"は、原形を特定できなかった時でも正しく処理できるように
            parsed_words.append(word)

名詞,代名詞,一般,*
<class 'list'>
名詞
助詞,係助詞,*,*
<class 'list'>
助詞
動詞,自立,*,*
<class 'list'>
動詞
助詞,接続助詞,*,*
<class 'list'>
助詞
動詞,非自立,*,*
<class 'list'>
動詞
名詞,一般,*,*
<class 'list'>
名詞
助詞,格助詞,一般,*
<class 'list'>
助詞
動詞,自立,*,*
<class 'list'>
動詞
助動詞,*,*,*
<class 'list'>
助動詞
助動詞,*,*,*
<class 'list'>
助動詞


In [ ]:
from janome.tokenizer import Tokenizer
import os

INPUT_FILE = 'soseki_corpus_preprocessed.txt' # 前処理済みファイル
OUTPUT_FILE = 'soseki_corpus_tokenized.txt' # Word2Vec学習用コーパスの保存先

TARGET_POS = ('名詞', '動詞', '形容詞', '副詞')

try:
    t = Tokenizer()
    with open(INPUT_FILE, 'r', encoding='utf-8') as f:
        cleaned_text = f.read()

    print(f"テキストの読み込み完了。総文字数: {len(cleaned_text)}")

    parsed_words = []

    for token in t.tokenize(cleaned_text):#t.tokenizeで、形態素に分解
        pos = token.part_of_speech.split(',')[0]

        if pos in TARGET_POS:
            word = token.base_form

            if word and word != '*':
                parsed_words.append(word)

    print(f"形態素解析とフィルタリング完了。総単語数: {len(parsed_words)}")

    # 4. 最終コーパスの書き出し
    # Word2Vecが読み込めるように、単語を半角スペース区切りで結合し、ファイルに保存
    final_corpus = ' '.join(parsed_words)

    with open(OUTPUT_FILE, 'w', encoding='utf-8') as f:
        f.write(final_corpus)

    print(f"最終コーパスを '{OUTPUT_FILE}' に保存しました。")
    print(f"次はこのファイルをWord2Vecモデルの学習に使用します。")

except FileNotFoundError:
    print(f"🚨 エラー: '{INPUT_FILE}' が見つかりませんでした。ファイル名を確認してください。")



テキストの読み込み完了。総文字数: 2479119
形態素解析とフィルタリング完了。総単語数: 808219
最終コーパスを 'soseki_corpus_tokenized.txt' に保存しました。
次はこのファイルをWord2Vecモデルの学習に使用します。
